In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import pathlib

## Loading data

In [2]:
directory_path = '../../04_-_Dev/videos'
features = 'emobase'#''

In [7]:
currentDirectory = pathlib.Path(directory_path)
currentPattern = "*." + features + ".annotated.csv"
file_list = [str(currentFile) for currentFile in currentDirectory.glob(currentPattern)]

df_total = pd.DataFrame()
for filename in file_list:
    df = pd.read_csv(filename, delimiter=';')
    df_total = pd.concat([df_total, df], axis=0)

print('Number of videos', df_total['video_name'].nunique())
print('Number of annotations', df_total[['video_name','diapo']].drop_duplicates().shape[0])


Number of videos 30
Number of annotations 240


In [8]:
df_total = df_total.dropna(axis=0)

In [9]:
df_total.to_pickle(directory_path + '/audio_' + features + '_data.p')

In [3]:
df_total = pd.read_pickle(directory_path + '/audio_' + features + '_data.p')

## Data pre-processing 1 - Sans utilisation de la temporalité

In [11]:
df_total

,frameIndex,frameTime,pcm_intensity_sma,pcm_loudness_sma,mfcc_sma[1],mfcc_sma[2],mfcc_sma[3],mfcc_sma[4],mfcc_sma[5],mfcc_sma[6],...,pcm_zcr_sma_de,voiceProb_sma_de,F0_sma_de,F0env_sma_de,video_name,stress_global,type_candidat,sexe,stress,diapo
0,0,0.00,0.000000e+00,0.000000,-1.839101,2.378769,-2.217382,1.524796,-0.692341,0.101261,...,-0.000242,0.016226,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
1,1,0.01,0.000000e+00,0.000000,-1.839101,2.378769,-2.217382,1.524796,-0.692341,0.101261,...,-0.000393,0.007565,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
2,2,0.02,0.000000e+00,0.000000,-5.952965,6.019684,-3.599674,1.358609,-0.693720,0.796165,...,-0.000272,0.011002,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
3,3,0.03,0.000000e+00,0.000000,-4.113864,3.640915,-1.382292,-0.166187,-0.001379,0.694905,...,0.000000,0.009354,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
4,4,0.04,0.000000e+00,0.000000,-5.911817,5.975944,-3.576790,1.367325,-0.725028,0.827764,...,0.000272,0.007488,0.0,0.0,WIN_20210409_10_26_11_Pro,0.0,Stagiaire,F,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78139,78139,781.39,8.054489e-12,0.029190,-19.760860,6.082466,-2.668031,5.229023,3.715201,-1.398331,...,-0.002025,0.012437,0.0,0.0,WIN_20210416_08_06_54_Pro,1.0,Stagiaire,F,1.0,18
78140,78140,781.40,4.319410e-12,0.023441,-21.733470,5.059051,-2.391827,4.356356,5.255848,-0.879569,...,-0.003778,0.021489,0.0,0.0,WIN_20210416_08_06_54_Pro,1.0,Stagiaire,F,1.0,18
78141,78141,781.41,3.975198e-12,0.022989,-21.913380,3.731894,-4.589119,0.219700,1.376662,-5.050643,...,-0.005409,0.040231,0.0,0.0,WIN_20210416_08_06_54_Pro,1.0,Stagiaire,F,1.0,18
78142,78142,781.42,3.662190e-12,0.022472,-22.256340,2.845562,-4.491580,2.111024,3.834707,-3.251046,...,-0.007011,0.052342,0.0,0.0,WIN_20210416_08_06_54_Pro,1.0,Stagiaire,F,1.0,18


In [12]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

In [13]:
# Sequence as a whole with 9 agregators
X = df_total.iloc[:,2:].groupby(['video_name','diapo']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-18]

In [14]:
# Sequence as a whole with 4 agregators
#X = df_total.iloc[:,2:].groupby(['video_name','diapo']).agg({'mean','std', kurtosis, skew}).iloc[:,:-8]

In [15]:
X.shape

(240, 468)

In [16]:
X

pcm_intensity_sma                             \
                                           median   percentil25     kurtosis   
video_name                diapo                                                
Test_pour_AFPA            1          6.973596e-11  3.821638e-11  3147.223384   
                          8          1.126761e-08  7.489974e-11   123.403605   
                          9          2.440317e-09  6.744114e-11   806.978682   
                          10         7.152661e-10  5.894842e-11   796.376223   
                          11         5.700818e-09  7.155228e-11   827.561300   
...                                           ...           ...          ...   
WIN_20210417_14_53_12_Pro 10         2.086603e-13  0.000000e+00    57.749166   
                          11         6.047051e-11  0.000000e+00    38.423855   
                          12         2.128377e-12  1.669329e-13   151.489483   
                          17         2.809062e-08  4.173518e-14    28.893323   
                          18         3.088259e-12  0.000000e+00  1139.092180   

                                                                           \
                                          max           std           min   
video_name                diapo                                             
Test_pour_AFPA            1      1.795643e-05  2.748470e-07  3.338736e-13   
                          8      2.552147e-05  1.159030e-06  2.629277e-12   
                          9      4.923681e-05  1.142409e-06  1.293720e-12   
                          10     5.260212e-05  1.367874e-06  1.544100e-12   
                          11     6.543399e-05  1.446588e-06  8.346372e-13   
...                                       ...           ...           ...   
WIN_20210417_14_53_12_Pro 10     6.300619e-06  4.197754e-07  0.000000e+00   
                          11     5.826240e-06  3.674674e-07  0.000000e+00   
                          12     9.125553e-09  3.760241e-10  0.000000e+00   
                          17     8.491340e-06  7.997970e-07  0.000000e+00   
                          18     1.093041e-08  2.102153e-10  0.000000e+00   

                                                                        \
                                         mean   percentil75       skew   
video_name                diapo                                          
Test_pour_AFPA            1      1.137963e-08  1.225284e-10  52.105435   
                          8      3.445894e-07  1.725128e-07   8.773747   
                          9      2.626492e-07  1.095324e-07  21.745491   
                          10     2.443886e-07  9.802079e-08  23.775476   
                          11     2.895271e-07  1.422769e-07  23.492521   
...                                       ...           ...        ...   
WIN_20210417_14_53_12_Pro 10     1.405645e-07  6.072907e-08   6.260282   
                          11     1.452637e-07  1.018110e-07   4.954644   
                          12     9.356331e-11  2.517556e-11  10.185306   
                          17     3.458074e-07  3.321533e-07   4.658821   
                          18     4.188290e-11  3.496205e-11  28.084717   

                                pcm_loudness_sma  ... F0_sma_de F0env_sma_de  \
                                          median  ...      skew       median   
video_name                diapo                   ...                          
Test_pour_AFPA            1             0.050654  ...  0.299397          0.0   
                          8             0.224705  ...  0.226955          0.0   
                          9             0.141097  ...  0.105952          0.0   
                          10            0.097588  ...  0.263135          0.0   
                          11            0.182440  ...  0.101022          0.0   
...                                          ...  ...       ...          ...   
WIN_20210417_14_53_12_Pro 10            0.006974  ... -0.001834          0.

In [17]:
X_temp = df_total[['video_name','diapo','type_candidat','sexe']].groupby(['video_name','diapo']).agg({'type_candidat':'first','sexe':'first'})
X_temp.loc[X_temp['type_candidat'] == 'Stagiaire','Stagiaire'] = 1
X_temp.loc[X_temp['type_candidat'] != 'Stagiaire','Stagiaire'] = 0
X_temp.loc[X_temp['sexe'] == 'F','Femme'] = 1
X_temp.loc[X_temp['sexe'] != 'F','Femme'] = 0
X_temp = X_temp.drop(['type_candidat','sexe'], axis=1)

In [18]:
X_temp

Stagiaire  Femme
video_name                diapo                  
Test_pour_AFPA            1            1.0    0.0
                          8            1.0    0.0
                          9            1.0    0.0
                          10           1.0    0.0
                          11           1.0    0.0
...                                    ...    ...
WIN_20210417_14_53_12_Pro 10           1.0    1.0
                          11           1.0    1.0
                          12           1.0    1.0
                          17           1.0    1.0
                          18           1.0    1.0

[240 rows x 2 columns]

In [19]:
X = pd.concat([X,X_temp],axis=1)

In [20]:
y = df_total.iloc[:,2:].groupby(['video_name','diapo']).agg({'stress':'mean'}).iloc[:,-1]

In [21]:
X.shape, y.shape

((240, 470), (240,))

In [22]:
X.to_pickle(directory_path + '/audio_' + features + '_data_X.p')
y.to_pickle(directory_path + '/audio_' + features + '_data_y.p')

In [23]:
from scipy.stats import kurtosis, skew
X_audio = df_total.loc[df_total['diapo'].isin([8, 9, 10, 11, 17]),:].iloc[:,2:].groupby(['video_name','diapo']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-18]
y_audio = df_total.loc[df_total['diapo'].isin([8, 9, 10, 11, 17]),:].iloc[:,2:].groupby(['video_name','diapo']).agg({'stress':'mean'}).iloc[:,-1]

X_audio.to_pickle(directory_path + '/audio_' + features + '_data_X_audio.p')
y_audio.to_pickle(directory_path + '/audio_' + features + '_data_y_audio.p')

In [24]:
X_audio.shape

(150, 468)

## Modèles

In [3]:
X = pd.read_pickle(directory_path + '/audio_' + features + '_data_X.p')
y = pd.read_pickle(directory_path + '/audio_' + features + '_data_y.p')

X_audio = pd.read_pickle(directory_path + '/audio_' + features + '_data_X_audio.p')
y_audio = pd.read_pickle(directory_path + '/audio_' + features + '_data_y_audio.p')

In [4]:
X.shape, X_audio.shape

((240, 470), (150, 468))

In [27]:
X

(pcm_intensity_sma, median)  \
video_name                diapo                                
Test_pour_AFPA            1                     6.973596e-11   
                          8                     1.126761e-08   
                          9                     2.440317e-09   
                          10                    7.152661e-10   
                          11                    5.700818e-09   
...                                                      ...   
WIN_20210417_14_53_12_Pro 10                    2.086603e-13   
                          11                    6.047051e-11   
                          12                    2.128377e-12   
                          17                    2.809062e-08   
                          18                    3.088259e-12   

                                 (pcm_intensity_sma, percentil25)  \
video_name                diapo                                     
Test_pour_AFPA            1                          3.821638e-11   
                          8                          7.489974e-11   
                          9                          6.744114e-11   
                          10                         5.894842e-11   
                          11                         7.155228e-11   
...                                                           ...   
WIN_20210417_14_53_12_Pro 10                         0.000000e+00   
                          11                         0.000000e+00   
                          12                         1.669329e-13   
                          17                         4.173518e-14   
                          18                         0.000000e+00   

                                 (pcm_intensity_sma, kurtosis)  \
video_name                diapo                                  
Test_pour_AFPA            1                        3147.223384   
                          8                         123.403605   
                          9                         806.978682   
                          10                        796.376223   
                          11                        827.561300   
...                                                        ...   
WIN_20210417_14_53_12_Pro 10                         57.749166   
                          11                         38.423855   
                          12                        151.489483   
                          17                         28.893323   
                          18                       1139.092180   

                                 (pcm_intensity_sma, max)  \
video_name                diapo                             
Test_pour_AFPA            1                  1.795643e-05   
                          8                  2.552147e-05   
                          9                  4.923681e-05   
                          10                 5.260212e-05   
                          11                 6.543399e-05   
...                                                   ...   
WIN_20210417_14_53_12_Pro 10                 6.300619e-06   
                          11                 5.826240e-06   
                          12                 9.125553e-09   
                          17                 8.491340e-06   
                          18                 1.093041e-08   

                                 (pcm_intensity_sma, std)  \
video_name                diapo                             
Test_pour_AFPA            1                  2.748470e-07   
                          8                  1.159030e-06   
                          9                  1.142409e-06   
                          10                 1.367874e-06   
                          11                 1.446588e-06   
...                                                   ...   
WIN_20210417_14_53_12_Pro 10                 4.197754e-07   
                          11                 3.674674e-07   
                          12           

In [28]:
X = X.fillna(0)

### Leave ont interview out
#### All diapos

In [29]:
parameters = {'n_estimators': [100, 150, 200], 'max_depth':[5, 6, 7, 8, 9,10, 11, 12, 13, 14, 15, 20, 25]}
parameters = {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20, 25]}

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)
X_no_name = X
y_no_name = y

model = RandomForestClassifier(random_state = 42, n_jobs=-1)

clf = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=cv_loo, verbose=1)
clf.fit(X_no_name, y_no_name)
print('Best results', clf.best_score_)
print('Best params', clf.best_params_)

Fitting 30 folds for each of 32 candidates, totalling 960 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 960 out of 960 | elapsed:  3.2min finished
Best results 0.5083333333333333
Best params {'max_depth': 6, 'n_estimators': 150}


In [30]:
print('accuracy (mean, std)', clf.cv_results_['mean_test_accuracy_score'][clf.best_index_], clf.cv_results_['std_test_accuracy_score'][clf.best_index_])
print('f1 (mean, std)', clf.cv_results_['mean_test_f1_score'][clf.best_index_], clf.cv_results_['std_test_f1_score'][clf.best_index_])

accuracy (mean, std) 0.5083333333333333 0.24352731993670762
f1 (mean, std) 0.4829673798423798 0.2621012974275235


In [7]:
parameters = {'n_estimators': [100, 150, 200], 'max_depth':[5, 6, 7, 8, 9,10, 11, 12, 13, 14, 15, 20, 25]}
parameters = {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20, 25]}

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)
X_no_name = X
y_no_name = y

model = BalancedRandomForestClassifier(random_state = 42, n_jobs=-1)

clf = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=cv_loo, verbose=1)
clf.fit(X_no_name, y_no_name)
print('Best results', clf.best_score_)
print('Best params', clf.best_params_)
print('accuracy (mean, std)', clf.cv_results_['mean_test_accuracy_score'][clf.best_index_], clf.cv_results_['std_test_accuracy_score'][clf.best_index_])
print('f1 (mean, std)', clf.cv_results_['mean_test_f1_score'][clf.best_index_], clf.cv_results_['std_test_f1_score'][clf.best_index_])

Fitting 30 folds for each of 32 candidates, totalling 960 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Best results 0.1625
Best params {'max_depth': 10, 'n_estimators': 50}
accuracy (mean, std) 0.1625 0.17425436388605406
f1 (mean, std) 0.18068376068376066 0.18754654977364366
[Parallel(n_jobs=1)]: Done 960 out of 960 | elapsed:  3.3min finished


In [31]:
# Getting predictions with a leave one interview out
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

#model = RandomForestClassifier(random_state = 42, n_jobs=-1, max_depth= 10, n_estimators= 50)
model = clf.best_estimator_

ytest_predict = np.zeros(len(y))
for train_index, test_index in loo.split(X, y, groups):
    #print(train_index, test_index)
    Xtrain, Xtest = X.iloc[train_index], X.iloc[test_index]
    ytrain, ytest = y.iloc[train_index], y.iloc[test_index]

    model.fit(Xtrain, ytrain)
    ytest_predict_temp = model.predict(Xtest)
    #print(ytest_predict_temp.shape)
    #print(test_index.shape)
    ytest_predict[test_index] = ytest_predict_temp
y_predict = ytest_predict

df_ypredict = pd.concat([X.reset_index()[['video_name','diapo']],pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_diapo.csv')

### Audio diapos only

In [5]:
parameters = {'n_estimators': [100, 150, 200], 'max_depth':[5, 6, 7, 8, 9,10, 11, 12, 13, 14, 15, 20, 25]}

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

groups = X_audio.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X_audio, y_audio, groups)
X_no_name = X_audio
y_no_name = y_audio

model = RandomForestClassifier(random_state = 42, n_jobs=-1)

clf = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=cv_loo)
clf.fit(X_no_name, y_no_name)
print('Best results', clf.best_score_)
print('Best params', clf.best_params_)

Best results 0.5133333333333333
Best params {'max_depth': 7, 'n_estimators': 100}


In [7]:
print('accuracy (mean, std)', clf.cv_results_['mean_test_accuracy_score'][clf.best_index_], clf.cv_results_['std_test_accuracy_score'][clf.best_index_])
print('f1 (mean, std)', clf.cv_results_['mean_test_f1_score'][clf.best_index_], clf.cv_results_['std_test_f1_score'][clf.best_index_])

accuracy (mean, std) 0.5133333333333333 0.3127654428197307
f1 (mean, std) 0.48170899470899464 0.3360067088800735


In [8]:
# Getting predictions with a leave one interview out
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
groups = X_audio.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X_audio, y_audio, groups)

#model = RandomForestClassifier(random_state = 42, n_jobs=-1, max_depth= 7, n_estimators= 100)
model = clf.best_estimator_ 

ytest_predict = np.zeros(len(y_audio))
for train_index, test_index in loo.split(X_audio, y_audio, groups):
    #print(train_index, test_index)
    Xtrain, Xtest = X_audio.iloc[train_index], X_audio.iloc[test_index]
    ytrain, ytest = y_audio.iloc[train_index], y_audio.iloc[test_index]

    model.fit(Xtrain, ytrain)
    ytest_predict_temp = model.predict(Xtest)
    #print(ytest_predict_temp.shape)
    #print(test_index.shape)
    ytest_predict[test_index] = ytest_predict_temp
y_predict = ytest_predict

df_ypredict = pd.concat([X_audio.reset_index()[['video_name','diapo']],pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_diapo_audio_only.csv')

## Stress global
### All diapos

In [35]:
df_ypredict = pd.read_csv('ypredict_' + features + '_diapo.csv')
df_ypredict = df_ypredict.drop(df_ypredict.columns[0],axis=1)
ypredict_stress_diapo = df_ypredict.pivot_table(values='ypredict', columns='diapo',index='video_name',aggfunc='mean')
#ypredict_stress_diapo

In [36]:
df_annotations_stress_diapo = df_total.pivot_table(values='stress', index='video_name',columns='diapo', aggfunc='mean')
df_annotations_stress_global = df_total.pivot_table(values='stress_global', index='video_name', aggfunc='mean')
df_annotations_stress = df_annotations_stress_diapo.merge(df_annotations_stress_global, on='video_name')
#df_annotations_stress

In [37]:
df_annotations_stress.to_csv('annotations.csv')

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

parameters = {'C': [0.01, 0.05, 0.1, 0.5, 1, 2], 'class_weight' : [None, 'balanced']}

X = df_annotations_stress.iloc[:,:-1]
#X = ypredict_stress_diapo
y = df_annotations_stress.iloc[:,-1]

model = LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42)
clf = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=5, verbose=1)
clf.fit(X, y)
print('Best results', clf.best_score_)
print('Best params', clf.best_params_)

# Prediction
clf.best_estimator_.fit(X,y)
ypredict_stress_global = clf.best_estimator_.predict(ypredict_stress_diapo)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Best results 0.8
Best params {'C': 0.1, 'class_weight': None}
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.9s finished


In [39]:
from sklearn.metrics import accuracy_score, f1_score
print('Accuracy',accuracy_score(y.values,ypredict_stress_global))
print('F1',f1_score(y.values,ypredict_stress_global, average='weighted'))

Accuracy 0.36666666666666664
F1 0.31666666666666665


In [40]:
pd.concat([y.reset_index(), pd.DataFrame(ypredict_stress_global,columns=['predicted_stress_global'])],axis=1)

,video_name,stress_global,predicted_stress_global
0,Test_pour_AFPA,1.0,1.0
1,Video_1,0.0,0.0
2,WIN_20210323_19_17_40_Pro,1.0,0.0
3,WIN_20210329_10_16_02_Pro,1.0,1.0
4,WIN_20210330_13_10_29_Pro,0.0,1.0
5,WIN_20210331_21_22_52_Pro,1.0,1.0
6,WIN_20210402_14_27_50_Pro,1.0,2.0
7,WIN_20210402_19_04_53_Pro,2.0,0.0
8,WIN_20210403_18_49_15_Pro,2.0,1.0
9,WIN_20210404_10_58_27_Pro,1.0,0.0


### Audio diapos only

In [9]:
df_ypredict = pd.read_csv('ypredict_' + features + '_diapo_audio_only.csv')
df_ypredict = df_ypredict.drop(df_ypredict.columns[0],axis=1)
df_ypredict.columns = ['video_name','diapo','ypredict']
ypredict_stress_diapo = df_ypredict.pivot_table(values='ypredict', columns='diapo',index='video_name',aggfunc='mean')

In [10]:
ypredict_stress_diapo

diapo,8,9,10,11,17
video_name,,,,,
Test_pour_AFPA,1.0,1.0,0.0,1.0,0.0
Video_1,0.0,0.0,0.0,0.0,1.0
WIN_20210323_19_17_40_Pro,1.0,1.0,1.0,1.0,0.0
WIN_20210329_10_16_02_Pro,1.0,1.0,1.0,1.0,1.0
WIN_20210330_13_10_29_Pro,1.0,1.0,1.0,1.0,2.0
WIN_20210331_21_22_52_Pro,1.0,1.0,1.0,1.0,1.0
WIN_20210402_14_27_50_Pro,2.0,2.0,2.0,2.0,2.0
WIN_20210402_19_04_53_Pro,1.0,1.0,1.0,1.0,1.0
WIN_20210403_18_49_15_Pro,1.0,1.0,1.0,1.0,1.0


In [11]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress[['video_name','8','9','10','11','17','stress_global']].set_index('video_name')
df_annotations_stress

,8,9,10,11,17,stress_global
video_name,,,,,,
Test_pour_AFPA,1.0,0.0,0.0,0.0,0.0,1.0
Video_1,0.0,0.0,0.0,1.0,1.0,0.0
WIN_20210323_19_17_40_Pro,0.0,1.0,1.0,1.0,1.0,1.0
WIN_20210329_10_16_02_Pro,0.0,1.0,1.0,0.0,0.0,1.0
WIN_20210330_13_10_29_Pro,0.0,0.0,0.0,1.0,0.0,0.0
WIN_20210331_21_22_52_Pro,0.0,1.0,1.0,1.0,0.0,1.0
WIN_20210402_14_27_50_Pro,1.0,1.0,2.0,1.0,1.0,1.0
WIN_20210402_19_04_53_Pro,1.0,1.0,1.0,1.0,1.0,2.0
WIN_20210403_18_49_15_Pro,1.0,1.0,1.0,1.0,2.0,2.0


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

parameters = {'C': [0.01, 0.05, 0.1, 0.5, 1, 2], 'class_weight' : [None, 'balanced']}

X = df_annotations_stress.iloc[:,:-1]
#X = ypredict_stress_diapo
y = df_annotations_stress.iloc[:,-1]

model = LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42)
clf = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=5, verbose=1)
clf.fit(X, y)
print('Best results', clf.best_score_)
print('Best params', clf.best_params_)

# Prediction
clf.best_estimator_.fit(X,y)
ypredict_stress_global = clf.best_estimator_.predict(ypredict_stress_diapo)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Best results 0.7666666666666667
Best params {'C': 0.1, 'class_weight': 'balanced'}
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.7s finished


In [13]:
from sklearn.metrics import accuracy_score, f1_score
print('Accuracy',accuracy_score(y.values,ypredict_stress_global))
print('F1',f1_score(y.values,ypredict_stress_global, average='weighted'))

Accuracy 0.5
F1 0.48967236467236464


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

parameters = {'max_depth': [2, 3, 4, 5], 'class_weight' : [None, 'balanced'], 'criterion': ['gini','entropy']}

X = df_annotations_stress.iloc[:,:-1]
y = df_annotations_stress.iloc[:,-1]

model = DecisionTreeClassifier(random_state=42)
clf = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=5, verbose=1)
clf.fit(X, y)
print('Best results', clf.best_score_)
print('Best params', clf.best_params_)

# Prediction
clf.best_estimator_.fit(X,y)
ypredict_stress_global = clf.best_estimator_.predict(ypredict_stress_diapo)
from sklearn.metrics import accuracy_score, f1_score
print('Accuracy',accuracy_score(y.values,ypredict_stress_global))
print('F1',f1_score(y.values,ypredict_stress_global, average='weighted'))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Best results 0.7000000000000001
Best params {'class_weight': None, 'criterion': 'gini', 'max_depth': 5}
Accuracy 0.43333333333333335
F1 0.39470899470899473
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.5s finished


In [42]:
X.columns[np.argsort(clf.best_estimator_.feature_importances_)]

Index([8, 11, 12, 17, 9, 1, 18, 10], dtype='object')

In [43]:
clf.best_estimator_.tree_

## Aggregation all frames within the video to predict the global stress

In [14]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

X = df_total.iloc[:,2:].groupby(['video_name']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-27]

In [15]:
y = df_total.iloc[:,2:].groupby(['video_name']).agg({'stress_global':'mean'}).iloc[:,-1]

In [16]:
X.columns

MultiIndex([('pcm_intensity_sma',        'mean'),
            ('pcm_intensity_sma',         'max'),
            ('pcm_intensity_sma', 'percentil25'),
            ('pcm_intensity_sma',         'min'),
            ('pcm_intensity_sma', 'percentil75'),
            ('pcm_intensity_sma',        'skew'),
            ('pcm_intensity_sma',    'kurtosis'),
            ('pcm_intensity_sma',         'std'),
            ('pcm_intensity_sma',      'median'),
            ( 'pcm_loudness_sma',        'mean'),
            ...
            (        'F0_sma_de',      'median'),
            (     'F0env_sma_de',        'mean'),
            (     'F0env_sma_de',         'max'),
            (     'F0env_sma_de', 'percentil25'),
            (     'F0env_sma_de',         'min'),
            (     'F0env_sma_de', 'percentil75'),
            (     'F0env_sma_de',        'skew'),
            (     'F0env_sma_de',    'kurtosis'),
            (     'F0env_sma_de',         'std'),
            (     'F0env_sma_de', 

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

parameters = {'max_depth': [2, 3, 4, 5], 'class_weight' : [None, 'balanced'], 'criterion': ['gini','entropy']}

model = DecisionTreeClassifier(random_state=42)
model = RandomForestClassifier(random_state=42)
clf = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=5, verbose=1)
clf.fit(X, y)
print('Best results', clf.best_score_)
print('Best params', clf.best_params_)

# Prediction
clf.best_estimator_.fit(X,y)
#ypredict_stress_global = clf.best_estimator_.predict(ypredict_stress_diapo)
#from sklearn.metrics import accuracy_score, f1_score
#print('Accuracy',accuracy_score(y.values,ypredict_stress_global))
#print('F1',f1_score(y.values,ypredict_stress_global, average='weighted'))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    9.0s finished
Best results 0.4
Best params {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3}


RandomForestClassifier(class_weight='balanced', max_depth=3, random_state=42)

In [22]:
print('accuracy (mean, std)', clf.cv_results_['mean_test_accuracy_score'][clf.best_index_], clf.cv_results_['std_test_accuracy_score'][clf.best_index_])
print('f1 (mean, std)', clf.cv_results_['mean_test_f1_score'][clf.best_index_], clf.cv_results_['std_test_f1_score'][clf.best_index_])

accuracy (mean, std) 0.4 0.13333333333333333
f1 (mean, std) 0.3514285714285715 0.1315665252520314


In [23]:
X.columns[np.argsort(clf.best_estimator_.feature_importances_)][0:20]

MultiIndex([(   'pcm_intensity_sma',        'mean'),
            (           'F0env_sma',         'min'),
            (           'F0env_sma', 'percentil75'),
            (           'F0env_sma',         'std'),
            (        'F0env_sma_de',         'std'),
            ('pcm_intensity_sma_de',        'mean'),
            ('pcm_intensity_sma_de', 'percentil25'),
            ('pcm_intensity_sma_de',         'min'),
            ('pcm_intensity_sma_de', 'percentil75'),
            ('pcm_intensity_sma_de',      'median'),
            ( 'pcm_loudness_sma_de',        'mean'),
            ( 'pcm_loudness_sma_de',    'kurtosis'),
            ( 'pcm_loudness_sma_de',         'std'),
            ( 'pcm_loudness_sma_de',      'median'),
            (      'mfcc_sma_de[1]',        'skew'),
            (      'mfcc_sma_de[1]',    'kurtosis'),
            (      'mfcc_sma_de[1]',         'std'),
            (      'mfcc_sma_de[1]',      'median'),
            (      'mfcc_sma_de[5]',         '

### PCA + Reg Log

In [33]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score


pca = PCA()
# set the tolerance to a large value to make the example faster
logistic = LogisticRegression()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

parameters = {
    'pca__n_components': [1, 2, 3, 5, 10],
    'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 20],
}

clf = GridSearchCV(estimator=pipe, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=5, verbose=1)
clf.fit(X, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Fitting 5 folds for each of 40 candidates, totalling 200 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Best parameter (CV score=0.400):
{'logistic__C': 21.54434690031882, 'pca__n_components': 10}
[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:    6.0s finished


In [34]:
print('accuracy (mean, std)', clf.cv_results_['mean_test_accuracy_score'][clf.best_index_], clf.cv_results_['std_test_accuracy_score'][clf.best_index_])
print('f1 (mean, std)', clf.cv_results_['mean_test_f1_score'][clf.best_index_], clf.cv_results_['std_test_f1_score'][clf.best_index_])

accuracy (mean, std) 0.39999999999999997 0.16996731711975951
f1 (mean, std) 0.3246031746031746 0.20411488769115568
